In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','available_qty':'{:,}','volbuy':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}','sell_price':'{:.2f}','diff':'{:.2f}','cost':'{:.2f}',
    'amount':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
    'buy_amt':'{:,.2f}','sell_amt':'{:,.2f}','pct':'{:,.2f}%'
}
cols = 'trade name qty price reason market active'.split()

today = date.today()
today

datetime.date(2022, 3, 12)

### If process on Saturday

In [2]:
today = today - timedelta(days=1)
eow = today
bow = eow - timedelta(days=4)
(bow, eow)

(datetime.date(2022, 3, 7), datetime.date(2022, 3, 11))

### Restart and Run All Cells

In [3]:
sql = '''
SELECT name, buys.date AS buy_date, sells.date AS sell_date,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE sells.date BETWEEN '%s' AND '%s'
ORDER BY name, sells.date'''
sql = sql % (bow, eow)
print(sql)


SELECT name, buys.date AS buy_date, sells.date AS sell_date,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE sells.date BETWEEN '2022-03-07' AND '2022-03-11'
ORDER BY name, sells.date


In [4]:
sells_df = pd.read_sql(sql, conpf)
sells_df.style.format(format_dict)

,name,buy_date,sell_date,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,profit
0,DOHOME,2022-01-05,2022-03-10,22.50,20.25,2.25,"4,800","108,000.00","97,200.00","10,800.00",11.11%,"10,345.51"
1,MEGA,2022-02-15,2022-03-09,43.25,43.00,0.25,"6,000","259,500.00","258,000.00","1,500.00",0.58%,353.79
2,SINGER,2022-02-14,2022-03-10,48.50,42.25,6.25,"1,500","72,750.00","63,375.00","9,375.00",14.79%,"9,073.50"


In [5]:
sells_grp = sells_df.groupby(['name'])[['qty','sell_amt','buy_amt','gross','profit']].sum()
sells_grp.style.format(format_dict)

,qty,sell_amt,buy_amt,gross,profit
name,,,,,
DOHOME,"4,800","108,000.00","97,200.00","10,800.00","10,345.51"
MEGA,"6,000","259,500.00","258,000.00","1,500.00",353.79
SINGER,"1,500","72,750.00","63,375.00","9,375.00","9,073.50"


In [6]:
sells_df[['sell_amt','buy_amt','gross','profit']].sum()

sell_amt    440250.0
buy_amt     418575.0
gross        21675.0
profit       19772.8
dtype: float64

In [7]:
sql = '''
SELECT name, buys.date AS buy_date, buys.price AS buy_price, 
qty, (buys.price * qty) AS buy_amt 
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE buys.date BETWEEN '%s' AND '%s'
ORDER BY name, buys.date'''
sql = sql % (bow, eow)
print(sql)


SELECT name, buys.date AS buy_date, buys.price AS buy_price, 
qty, (buys.price * qty) AS buy_amt 
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE buys.date BETWEEN '2022-03-07' AND '2022-03-11'
ORDER BY name, buys.date


In [8]:
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,buy_date,buy_price,qty,buy_amt
0,AIMIRT,2022-03-10,13.00,"10,000","130,000.00"
1,ASP,2022-03-07,3.80,"30,000","114,000.00"
2,TPIPL,2022-03-07,1.70,"60,000","102,000.00"
3,WHART,2022-03-07,12.10,"10,000","121,000.00"


In [9]:
buys_df[['buy_amt']].sum()

buy_amt    467000.0
dtype: float64

In [10]:
sells_df[['buy_amt']].sum()

buy_amt    418575.0
dtype: float64

In [11]:
diff_amt = buys_df[['buy_amt']].sum() - sells_df[['buy_amt']].sum()
diff_amt

buy_amt    48425.0
dtype: float64

In [12]:
sql = '''
SELECT name, available_qty, cost, status
FROM stocks
'''
stocks_df = pd.read_sql(sql, conlite)
stocks_df.set_index('name', inplace=True)
stocks_df.style.format(format_dict)

,available_qty,cost,status
name,,,
MCS,"50,000",16.70,T
PTTGC,"12,000",58.75,T
JASIF,"110,000",10.10,I
DIF,"50,000",14.70,T
ROJNA,0,6.35,X
HREIT,"40,100",8.95,I
MEGA,"-1,500",43.00,O
JMT,0,40.50,O
GULF,0,0.00,O


In [13]:
df_merge = pd.merge(sells_grp,stocks_df,on='name',how='inner')
df_merge.style.format(format_dict)

,qty,sell_amt,buy_amt,gross,profit,available_qty,cost,status
name,,,,,,,,
DOHOME,"4,800","108,000.00","97,200.00","10,800.00","10,345.51","9,600",20.25,I
MEGA,"6,000","259,500.00","258,000.00","1,500.00",353.79,"-1,500",43.00,O
SINGER,"1,500","72,750.00","63,375.00","9,375.00","9,073.50","3,000",42.25,I


In [14]:
sql = '''
SELECT name, SUM(volbuy) AS volbuy, price
FROM buy
WHERE active = 1
GROUP BY name
ORDER BY name'''
buy_df = pd.read_sql(sql, const)
buy_df.volbuy = buy_df.volbuy.astype(int)
buy_df.style.format(format_dict)

,name,volbuy,price
0,AIMIRT,"10,000",13.00
1,ASP,"30,000",3.80
2,BCH,"15,000",21.30
3,CPNCG,"20,000",12.40
4,DIF,"50,000",14.70
5,DOHOME,"9,600",20.25
6,HREIT,"40,100",8.95
7,IP,"7,500",20.20
8,JASIF,"110,000",10.10
9,KCE,"11,000",75.50


In [15]:
df_merge2 = pd.merge(buy_df,stocks_df,on='name',how='inner')
df_merge2.style.format(format_dict)

,name,volbuy,price,available_qty,cost,status
0,AIMIRT,"10,000",13.00,"10,000",13.00,I
1,ASP,"30,000",3.80,"30,000",3.80,I
2,BCH,"15,000",21.30,"15,000",21.30,I
3,CPNCG,"20,000",12.40,"20,000",12.40,I
4,DIF,"50,000",14.70,"50,000",14.70,T
5,DOHOME,"9,600",20.25,"9,600",20.25,I
6,HREIT,"40,100",8.95,"40,100",8.95,I
7,IP,"7,500",20.20,"7,500",20.20,I
8,JASIF,"110,000",10.10,"110,000",10.10,I
9,KCE,"11,000",75.50,"11,000",75.50,U


In [16]:
mask1 = df_merge2.volbuy != df_merge2.available_qty
mask2 = df_merge2.price != df_merge2.cost
df_merge2[mask1 | mask2].style.format(format_dict)

,name,volbuy,price,available_qty,cost,status
